In [50]:
import pandas as pd 
import numpy as np

train = pd.read_csv('water_train.csv').drop('id', axis= 1).sample(n =20000, random_state=2)
test = pd.read_csv('water_train_label.csv').drop('id', axis= 1).sample(n =20000, random_state=2)
print(train.shape, test.shape)

(20000, 39) (20000, 1)


In [51]:
test.status_group.value_counts()

status_group
functional                 10851
non functional              7698
functional needs repair     1451
Name: count, dtype: int64

In [52]:
train.head()

,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,...,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group
2099,0.0,2013-02-07,Fini Water,48,Fini water,40.177347,-10.483661,Kwa Namalili,0,Ruvuma / Southern Coast,...,unknown,milky,milky,enough,enough,shallow well,shallow well,groundwater,other,other
13681,0.0,2013-02-22,Jaica,1474,JAICA,34.563707,-1.722762,Baru,0,Lake Victoria,...,unknown,salty,salty,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump
55259,0.0,2013-03-17,Wfp,1909,Active MKM,35.369368,-1.867549,Shuleni,0,Lake Victoria,...,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe
2649,500.0,2011-02-21,Adb,1772,DWE,34.865276,-9.045687,Dispensary,0,Rufiji,...,monthly,soft,good,enough,enough,hand dtw,borehole,groundwater,hand pump,hand pump
40353,0.0,2011-03-05,Tabea,46,DWE,38.758088,-6.781934,none,0,Wami / Ruvu,...,never pay,salty,salty,insufficient,insufficient,river,river/lake,surface,hand pump,hand pump


In [53]:
from sklearn.model_selection import train_test_split

X_full_train, X_test, y_full_train, y_test = train_test_split(train, test, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_full_train, y_full_train, test_size=0.25, random_state=42)

In [54]:
# import dictvectorizer from sklearn.

from sklearn.feature_extraction import DictVectorizer

dv = DictVectorizer(sparse=False)
X_train_dict = X_train.to_dict(orient='records')
X_val_dict = X_test.to_dict(orient='records')
X_test_dict = X_test.to_dict(orient='records')

X_train_dv = dv.fit_transform(X_train_dict)
X_val_dv = dv.transform(X_val_dict)
X_test_dv = dv.transform(X_test_dict)

In [55]:
dv.get_feature_names_out()

array(['amount_tsh', 'basin=Internal', 'basin=Lake Nyasa', ...,
       'wpt_name=Zoza', 'wpt_name=Zuleha', 'wpt_name=none'], dtype=object)

In [56]:
# import decision tree classifier from sklearn.tree

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier()
dt.fit(X_train_dv, y_train)
y_pred = dt.predict(X_test_dv)
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_pred)

0.73925

In [62]:
# import gridsearch cv 
from sklearn.model_selection import GridSearchCV

param_grid = {'max_depth': [3, 5, 7, 9],
              'min_samples_split': [2, 5, 10],
              'min_samples_leaf': [1, 2, 4],}
grid_search = GridSearchCV(dt, param_grid, cv=3,error_score='raise')
grid_search.fit(X_train_dv, y_train)
grid_search.best_params_

accuracy_score(y_test, y_pred)

0.73925

In [63]:
grid_search.best_params_


{'max_depth': 9, 'min_samples_leaf': 1, 'min_samples_split': 5}